# ABS Monthly Business Turnover Indicator 

## Set-up

In [1]:
# analytic imports
import pandas as pd
import readabs as ra
from readabs import metacol as mc
from mgplot import seastrend_plot_finalise, series_growth_plot_finalise

# local imports
from abs_helper import get_abs_data

# display charts in this notebook
SHOW = False

## Get the data

In [2]:
catalog = "5681.0"
abs_dict, meta, source, RECENT = get_abs_data(catalog)
plot_times = None, RECENT
plot_tags = "", "-recent"
RFOOTER = f"ABS {catalog}"
#print(meta[mc.table].unique())

ValueError: _get_url(): no URL or valid catalogue number provided.

## And plot

In [ ]:
def headline_charts() -> None:
    """
    Plot headline charts for the ABS data.
    """

    # --- set-up
    table = '5681001_13-industry_summary'
    index_phrase = "Business Turnover Index ;  "
    dids = [x for x in meta[meta[mc.table] == table][mc.did].unique() if index_phrase in x]
    sa = "Seasonally Adjusted"
    trend = "Trend"
    data = abs_dict[table]

    for did in dids:
        # --- data extraction
        selector = {
            table: mc.table,
            did: mc.did,
            sa: mc.stype,
        }
        _table, sid, units = ra.find_abs_id(meta, selector, verbose=False)
        sa_data = data[sid]
        del selector[sa]
        selector[trend] = mc.stype
        _table, sid, units = ra.find_abs_id(meta, selector, verbose=False)
        trend_data = data[sid]

        # --- data plotting
        frame = pd.DataFrame(
            {
                sa: sa_data,
                trend: trend_data,
            },
        )
        title = did.replace(" Index ", "").replace(" ;  Current Price ;", "")
        seastrend_plot_finalise(
            frame,
            title=title,
            ylabel=index_phrase.replace(" ; ", ""),
            dropna=False,
            lfooter="Australia. Current Price.",
            rfooter=RFOOTER,
            show=SHOW,
        )

        series_growth_plot_finalise(
            frame[sa],
            plot_from=-15 ,
            title=title,
            lfooter=f"Australia. {sa}. Current Price.",
            rfooter=RFOOTER,
            tag="growth",
            legend=True,
            show=SHOW,
        )


headline_charts()

## The End